# EDS232/CalCOFI Ocean chemistry prediction 2025

Team members: Stephen Carroll, Tom Gibbons-Matsuyama, Ian Morris-Sibaja, Haylee Oyler 

### Description
For this quarter’s final lab, you will apply the machine learning knowledge you’ve gained to train models that predict dissolved inorganic carbon (DIC) levels in water samples collected by the California Cooperative Oceanic Fisheries Investigations program.

In this lab, you'll be working with real-world environmental data in a friendly competition with your classmates to see who can develop the most accurate predictive model.

## Your Task
- **Acquire domain knowledge**: Provided by Dr. Satterthwaite in her presentation
- **Explore the data**: Load the dataset and perform initial exploratory data analysis to inform your modeling choices
- **Preprocessing** (if necessary): Is the data ready to be used in your model?
- **Choose and train a model**: Select an appropriate machine learning algorithm for this task. Train your model on the provided training data
- **Tune relevant parameters**: Use cross-validation to optimize model performance. Experiment with different hyperparameters to reduce error
- **Submit your prediction**: Generate predictions on the provided test dataset

In [1]:
# Load libraries
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import statistics as stats
import time
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score
from scipy.stats import uniform, randint
from sklearn.impute import KNNImputer
from sklearn.tree import DecisionTreeRegressor, plot_tree
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder

# Import data
train_df = pd.read_csv("data/eds-232-cal-cofi-ocean-chemistry-prediction-2025/train.csv")
test_df = pd.read_csv("data/eds-232-cal-cofi-ocean-chemistry-prediction-2025/test.csv")
sample_sub =  pd.read_csv("data/eds-232-cal-cofi-ocean-chemistry-prediction-2025/sample_submission.csv")